In [108]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import surprise 
from sklearn.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import SVD
from IESEGRecSys import eval
from surprise import KNNBasic, SVD, CoClustering, Dataset, Reader

read tables

In [109]:
artist_df = pd.read_table('Data/Data Group Assignment/Artists.dat')
tags_df = pd.read_table('Data/Data Group Assignment/tags.dat',encoding='latin-1')
user_artists_df = pd.read_table('Data/Data Group Assignment/user_artists.dat')
user_taggedartists_df = pd.read_table('Data/Data Group Assignment/user_taggedartists.dat')


observe tables

In [110]:
tags_df

,tagID,tagValue
0,1,metal
1,2,alternative metal
2,3,goth rock
3,4,black metal
4,5,death metal
...,...,...
11941,12644,suomi
11942,12645,symbiosis
11943,12646,sverige
11944,12647,eire


In [111]:
user_artists_df.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [112]:
user_taggedartists_df['artistID'].nunique()

12523

## working on dataset for content based rec sys

In [113]:
#merging the dataset for content-based 
df_content_based = pd.merge(user_taggedartists_df,tags_df,on="tagID",how="inner")

In [128]:
#removing unnecessary variables + dropping duplicates for the remaining two columns
df_content_based= df_content_based[['artistID','tagValue']].sort_values(by="artistID").drop_duplicates()
df_content_based.head(10)


,artistID,tagValue
119352,1,japanese
66383,1,j-rock
120760,1,weeabo
75701,1,gothic
156127,1,better than lady gaga
66494,1,visual kei
142195,1,jrock
131349,2,vocal
75917,2,gothic rock
59020,2,seen live


In [115]:
#pivoting the table (creating a dummies table)
df_content_based_1 = df_content_based.pivot_table(index="artistID", columns = "tagValue", aggfunc={'tagValue':np.count_nonzero}).fillna("0")


In [116]:
df_content_based_1.head()

tagValue                                                              \
tagValue     '80s -pearl fashion music 0 play yet 00 00's 007 00s 00s rock  1   
artistID                                                                        
1               0                    0          0  0    0   0   0        0  0   
2               0                    0          0  0    0   0   0        0  0   
3               0                    0          0  0    0   0   0        0  0   
4               0                    0          0  0    0   0   0        0  0   
5               0                    0          0  0    0   0   0        0  0   

               ...                                                    \
tagValue 1008  ... zikirli zmiel pierogi zmierzch zombie zombie rave   
artistID       ...                                                     
1           0  ...       0             0        0      0           0   
2           0  ...       0             0        0      0           0   
3           0  ...       0             0        0      0           0   
4           0  ...       0             0        0      0           0   
5           0  ...       0             0        0      0           0   

                                            
tagValue zombieland zoocore zornish ztt zu  
artistID                                    
1                 0       0       0   0  0  
2                 0       0       0   0  0  
3                 0       0       0   0  0  
4                 0       0       0   0  0  
5                 0       0       0   0  0  

[5 rows x 9749 columns]

In [117]:
df_content_based_1.shape

(12523, 9749)

In [118]:
#non working code
# Creating dummy variables:
#df_content_based_1 = pd.get_dummies(df_content_based,prefix='', prefix_sep="" , columns=['tagValue'])
#df_content_based_1 = df_content_based_1.drop(['userID', 'artistID','day','month','year'], axis=1)
#df_content_based_1 = df_content_based_1.set_index("artistID")
#df_content_based_1.sort_values(by="tagID")


## collaborative filtering data prep

In [119]:
#discretize weights using qcuts 
user_artists_df['weight_quantiles'] = pd.qcut(user_artists_df['weight'],
                           q=[0,.2,.4,.6,.8,1],
                           labels=False,
                           precision=0)
user_artists_df=user_artists_df.drop(["weight"], axis = 1)

## Train test split

In [120]:
# train-test split
train, test = train_test_split(user_artists_df, test_size=0.3, random_state=123)

# reset index
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

print(train.shape)
print(test.shape)

(64983, 3)
(27851, 3)


## creating surprise datasets

In [121]:
reader = surprise.Reader(rating_scale=(1, 5))
df_train = surprise.Dataset.load_from_df(train, reader).build_full_trainset()
df_test = list(test.itertuples(index=False, name=None))

## Applying collaborative filtering models

In [129]:
# user-based
options = {'name':'cosine', 'user_based':True}
ub = KNNBasic(k=15, min_k=5, sim_options=options, random_state=42)

# item-based
options = {'name':'cosine', 'user_based':False}
ib = KNNBasic(k=15, min_k=5, sim_options=options, random_state=42)

# svd
mf = SVD(n_factors=20, biased=False, random_state=42)

# clustering
clust = CoClustering(n_cltr_u=10, n_cltr_i=10, n_epochs=50, random_state=42)


In [130]:
#evaluating the models
models = {"UB_15":ub, "IB_15":ib, "SVD_20":mf, "Clust_10_10":clust}
overview = pd.concat([eval.evaluate(mod.fit(df_train).test(df_test), topn=5, rating_cutoff=4) for mod in models.values()], axis=1)
overview.columns = list(models.keys())
overview

Computing the cosine similarity matrix...


C:\tools\Anaconda3\envs\py\lib\site-packages\surprise\prediction_algorithms\algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Computing the cosine similarity matrix...


C:\tools\Anaconda3\envs\py\lib\site-packages\surprise\prediction_algorithms\algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.


<ipython-input-130-a0315f584852>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  overview = pd.concat([eval.evaluate(mod.fit(df_train).test(df_test), topn=5, rating_cutoff=4) for mod in models.values()], axis=1)


,UB_15,IB_15,SVD_20,Clust_10_10
RMSE,1.393631,1.117180,1.217409,1.074960
MAE,1.183930,0.897713,0.968265,0.864092
Recall,0.000180,0.134892,0.080216,0.156115
Precision,0.250000,0.976562,0.867704,0.745704
F1,0.000359,0.237042,0.146855,0.258180
NDCG@5,0.866736,0.861017,0.868794,0.860111


In [123]:
print(len(df_content_based_1.columns))
print(len(pd.unique(df_content_based_1.columns)))

9749
9749


## Applying content based rec sys

In [124]:
from sklearn.metrics.pairwise import cosine_similarity
import surprise

# Content Based as a Class
class ContentBased:

    def __init__(self, NN):
        self.NN = NN
        self.fitted = {"content":False, "ratings":False}
        
    def fit(self, content_data):

        self.items = content_data.index.values
        self.item_dim = len(self.items)
        # check for duplicate items
        assert (len(self.items) == len(set(self.items))), "Duplicate items in content data!"

        # compute similarity
        self.matrix = cosine_similarity(content_data.values)
        np.fill_diagonal(self.matrix, 0)
        
        self.matrixNN = self.matrix.copy()

        # filter similarity matrix for NN nearest neighbors (constraint: non-negative similarity)
        for i in range(self.item_dim):
            crit_val = max(-np.sort(-self.matrix[i])[self.NN-1], 0)
            self.matrixNN[i][self.matrixNN[i] < crit_val] = 0.0
        
        self.fitted["content"] = True

    # helper -> transform surprise.trainset.Trainset to pd.DataFrame
    def _trainset2list(self, trainset):
        return pd.DataFrame([(trainset.to_raw_uid(u), trainset.to_raw_iid(i), r) for (u, i, r) in trainset.all_ratings()], columns=["user", "item", "rating"])

    def fit_ratings(self, df):

        if not self.fitted["content"]:
            raise Exception("Fit model on content data!")

        if isinstance(df, surprise.trainset.Trainset):
            df = self._trainset2list(df)
        
        # fix unknown items
        unknown_items = list(set(df["item"]) - set(self.items))
        if len(unknown_items) > 0:
            print(f"Warning {len(unknown_items)} items are not included in content data: {unknown_items}")
        df = df[df["item"].isin(self.items)].reset_index(drop=True)

        # store user data
        self.users = np.unique(df["user"])
        self.user_dim = len(self.users)

        # fix missing items
        missing_items = list(set(self.items) - set(df["item"]))
        if len(missing_items) > 0: 
            fix_df = pd.DataFrame([{"user":np.nan, "item":i, "rating":np.nan} for i in missing_items])
            df = df.append(fix_df).reset_index(drop=True)

        # pivot 
        df_pivot = df.pivot_table(index='user', values='rating', columns='item', dropna=False).reindex(self.users)

        # row-wise (user) average
        self.user_avg = np.array(np.mean(df_pivot, axis=1))
        self.global_mean = np.mean(self.user_avg)

        # center ratings
        df_pivot = df_pivot.sub(self.user_avg, axis=0).fillna(0)

        # predict ratings for each item 
        denom = self.matrixNN.sum(axis=0) # column sums
        self.prediction = (np.matmul(df_pivot.values, self.matrixNN) / denom) + self.user_avg[:,np.newaxis]

        # replace NA values with mean
        # prediction[np.isnan(prediction)] = self.global_mean

        self.fitted["ratings"] = True
    
    # get predicted value for user-item combination
    def predict(self, user, item, r_ui=None):
        details = {"was_impossible":False}

        # check whether user and item are unknown -> default = global average rating
        if self.knows_user(user) & self.knows_item(item):

            # convert user & item in internal ids
            iid = np.where(self.items == item)[0].item()
            uid = np.where(self.users == user)[0].item()

            # inference prediction
            est = self.prediction[uid, iid]
            
            if np.isnan(est): 
                est = self.global_mean
                details["was_impossible"] = True
            return surprise.Prediction(user, item, r_ui, est, details)
        
        else:
            details["was_impossible"] = True
            details["reason"] = "User or item unknown"
            return surprise.Prediction(user, item, r_ui, self.global_mean, details)

    # predict entire testset
    def test(self, testset):
        if not self.fitted["ratings"]:
            raise Exception("Fit model on ratings data!")
        return [self.predict(user=u,item=i,r_ui=r) for (u,i,r) in testset]

    def knows_user(self, user):
        return user in self.users   

    def knows_item(self, item):
        return item in self.items        

    # get topn most similar items 
    def get_most_similar(self, item, topn=5):

        # get iid
        if self.knows_item(item):
            iid = np.where(self.items == item)[0].item()
        else:
            raise Exception(f"Item {item} unknown ...")
        
        list_iids = (-self.matrix[iid]).argsort()[:topn]
        return self.items[list_iids]

    def get_similarities(self):
        print('Cosine similarities shape: ({}, {}) items x items'.format(self.item_dim, self.item_dim))
        return self.matrix

In [125]:
# init content-based
cb = ContentBased(NN=10)

# fit on content
cb.fit(df_content_based_1)

# fit on train_ratings
cb.fit_ratings(df_train)
    
# predict test ratings
cb_pred = cb.test(df_test)

Warning 4004 items are not included in content data: [16384, 8193, 8194, 16385, 8197, 16389, 16392, 16393, 8203, 8204, 16404, 21, 8213, 8220, 8227, 8228, 16419, 16420, 16424, 16426, 8235, 8236, 8237, 8238, 16428, 8240, 16429, 8242, 16439, 16443, 16444, 16445, 8257, 16450, 16451, 16457, 16461, 16462, 16469, 8281, 8282, 8284, 8289, 16490, 109, 110, 111, 8303, 16494, 8306, 115, 16496, 16507, 124, 8316, 8317, 127, 8320, 129, 8321, 131, 16508, 133, 8325, 135, 136, 8327, 8330, 139, 8332, 16518, 16519, 8335, 144, 145, 8338, 147, 149, 150, 8342, 8344, 16535, 8352, 8353, 8355, 8357, 168, 177, 16561, 8373, 8374, 16565, 16569, 16571, 16573, 16577, 16579, 16580, 8389, 16581, 16583, 16585, 8394, 16588, 16589, 16596, 16598, 16599, 16601, 8410, 8411, 16602, 8413, 16606, 224, 16615, 16618, 8427, 8428, 8430, 16623, 242, 8434, 8437, 16631, 8440, 16636, 254, 8447, 8448, 16638, 8450, 8451, 8452, 8453, 16640, 16642, 8456, 16645, 16648, 16649, 8460, 16651, 16653, 8463, 16654, 16655, 274, 16656, 16658, 277, 

<ipython-input-124-358ca43c89df>:71: RuntimeWarning: invalid value encountered in true_divide
  self.prediction = (np.matmul(df_pivot.values, self.matrixNN) / denom) + self.user_avg[:,np.newaxis]


In [126]:
# compute metrics for CB RS
cb_res = eval.evaluate(cb_pred, topn=5, rating_cutoff=4).rename(columns={'value':'Content_based_10'})
cb_res

,Content_based_10
RMSE,0.963876
MAE,0.730288
Recall,0.151079
Precision,0.927152
F1,0.259821
NDCG@5,0.870427


In [127]:
#get summary for all models applied
overview = pd.concat([overview, cb_res], axis=1)
overview

,UB_15,IB_15,SVD_20,Clust_10_10,Content_based_10
RMSE,1.393631,1.117180,1.217409,1.074960,0.963876
MAE,1.183930,0.897713,0.968265,0.864092,0.730288
Recall,0.000180,0.134892,0.080216,0.156115,0.151079
Precision,0.250000,0.976562,0.867704,0.745704,0.927152
F1,0.000359,0.237042,0.146855,0.258180,0.259821
NDCG@5,0.866736,0.861017,0.868794,0.860111,0.870427
